In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [14]:
df = pd.read_csv('data/final_train.csv')
test_df = pd.read_csv('data/final_test.csv')
test_id_df = pd.read_csv('data/test.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51635 entries, 0 to 51634
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id            51635 non-null  int64
 1   plate         51635 non-null  int64
 2   date          51635 non-null  int64
 3   price         51635 non-null  int64
 4   forbidden     51635 non-null  int64
 5   advantage     51635 non-null  int64
 6   significance  51635 non-null  int64
 7   city          51635 non-null  int64
dtypes: int64(8)
memory usage: 3.2 MB


In [6]:
X_train,y_train = df.drop(columns=['price']),df['price']

In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51635 entries, 0 to 51634
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id            51635 non-null  int64
 1   plate         51635 non-null  int64
 2   date          51635 non-null  int64
 3   forbidden     51635 non-null  int64
 4   advantage     51635 non-null  int64
 5   significance  51635 non-null  int64
 6   city          51635 non-null  int64
dtypes: int64(7)
memory usage: 2.8 MB


In [8]:
def smape(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Ensure same data type
    y_pred = tf.cast(y_pred, tf.float32)
    return 100 * tf.reduce_mean(2 * tf.abs(y_pred - y_true) / (tf.abs(y_true) + tf.abs(y_pred) + tf.keras.backend.epsilon()))

In [9]:
# Initialize the ANN
classifier = Sequential()

# Adding layers
classifier.add(Dense(units=50, kernel_initializer='he_uniform', activation='relu', input_dim=7))
classifier.add(Dense(units=25, kernel_initializer='he_uniform', activation='relu'))
classifier.add(Dense(units=50, kernel_initializer='he_uniform', activation='relu'))
classifier.add(Dense(units=1, kernel_initializer='he_uniform', activation='linear'))  # Linear activation for regression

# Compile the ANN with a proper loss function
classifier.compile(loss='mean_absolute_error', optimizer='Adamax', metrics=[smape])

# Fit the model
model_history = classifier.fit(X_train.values, y_train.values, validation_split=0.20, batch_size=10, epochs=100)

c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
4131/4131 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 381615.9375 - smape: 86.7791 - val_loss: 400768.4688 - val_smape: 81.6501
Epoch 2/100
4131/4131 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 374676.1562 - smape: 80.0391 - val_loss: 403666.3125 - val_smape: 82.4980
Epoch 3/100
4131/4131 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 367318.0000 - smape: 80.1688 - val_loss: 397684.0000 - val_smape: 80.6701
Epoch 4/100
4131/4131 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 361800.8438 - smape: 79.7097 - val_loss: 397155.2188 - val_smape: 80.4946
Epoch 5/100
4131/4131 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 374857.4375 - smape: 80.1477 - val_loss: 400242.3438 - val_smape: 81.4696
Epoch 6/100
4131/4131 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 374152.5000 - smape: 79.9502 - val_loss: 400535.6875 - val_smape: 81.5603
Epoch 7/100
4131/4131 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 367883.8125 - smape: 79.7572 - val_loss: 401477.9375 - val_smape: 81.8385
Epoch 8/100
4131/4131 ━━━━━━━━━━━━━━━━━━━

In [10]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['price']),df['price'],test_size = 0.2, random_state=2)

In [13]:
ann_pred=classifier.predict(test_df.values)

241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step


In [12]:
print(ann_pred)

[[171163.  ]
 [188090.48]
 [101714.31]
 ...
 [143949.7 ]
 [225557.33]
 [161756.95]]


In [16]:
pred=pd.DataFrame(ann_pred)
datasets=pd.concat([test_id_df['id'],pred],axis=1)
datasets.columns=['id','price']
datasets.to_csv('outputs/ann_sample_submission.csv',index=False)